In [1]:
pip install --upgrade transformers

     ---------------------------------------- 9.4/9.4 MB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 KB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 287.9/287.9 KB 1.3 MB/s eta 0:00:00
     -------------------------------------- 417.5/417.5 KB 1.1 MB/s eta 0:00:00
     -------------------------------------- 269.7/269.7 KB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\user\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
!pip install sentencepiece

You should consider upgrading via the 'C:\Users\user\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
pip show sentencepiece

Name: sentencepiece
Version: 0.2.0
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: c:\users\user\appdata\local\programs\python\python39\lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sentencepiece
print("SentencePiece imported successfully.")

In [8]:
from transformers import T5Tokenizer
try:
    tokenizer=T5Tokenizer.from_pretrained('t5-small')
    print("T5Tokenizer loaded successfully.")
except Exception as e:
    print(f"Error: {e}")

T5Tokenizer loaded successfully.


In [1]:
import torch
import torch.nn as nn

class LinearProjectionWithConv(nn.Module):
    def __init__(self, embed_dim):
        super(LinearProjectionWithConv, self).__init__()
        # Kernel size is (183, 1) and stride is (183, 1)
        self.conv = nn.Conv2d(in_channels=1, out_channels=embed_dim, kernel_size=(183, 1), stride=(183, 1))
    
    def forward(self, x):
        # x shape: (batch_size, 183, 1024)
        x = x.unsqueeze(1)  # Add channel dimension: (batch_size, 1, 183, 1024)
        x = self.conv(x)  # Apply convolution
        x = x.squeeze(2)  # Remove the channel dimension: (batch_size, embed_dim, new_sequence_length)
        return x


In [13]:
import os
import json
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer
from torch.utils.data import Dataset, DataLoader
from glob import glob

import json
import os

def  find_files(directory, pattern='**/*.json'):
    return glob(os.path.join(directory,pattern),recursive=True)

# Define paths to your folders
train_folder = 'D:/Vathsala/train_2D_keypoints/openpose_output/output'
val_folder = 'D:/Vathsala/val_2D_keypoints/openpose_output/output'
test_folder = 'D:/Vathsala/test_2D_keypoints/openpose_output/output'

# Load the datasets
train_data = find_files(train_folder)
val_data = find_files(val_folder)
test_data = find_files(test_folder)



# Tokenizer and Dataset Preparation
tokenizer = T5Tokenizer.from_pretrained('t5-small')
max_len = 512


class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze()
        }

# Create datasets and dataloaders

train_dataset = TextDataset(train_data, tokenizer, max_len)
val_dataset = TextDataset(val_data, tokenizer, max_len)
test_dataset = TextDataset(test_data, tokenizer, max_len)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)


In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class T5WithProjection(nn.Module):
    def __init__(self, t5_model_name, embed_dim):
        super(T5WithProjection, self).__init__()
        self.t5 = T5ForConditionalGeneration.from_pretrained('t5-small')
        self.projection = LinearProjectionWithConv(embed_dim)
    
    def forward(self, input_ids, attention_mask=None, labels=None):
         # Apply linear projection
        input_ids = self.projection(outputs.logits)  # Apply projection to logits
        # Forward pass through T5 model
        outputs = self.t5(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
       
        return outputs

# Initialize the model with projection
embed_dim = 1  # Adjust according to your T5 model's hidden size
model = T5WithProjection('t5-small', embed_dim)


In [16]:
import torch
from transformers import AdamW
from tqdm import tqdm
import torch.nn as nn
from transformers import T5ForConditionalGeneration, AdamW, T5Tokenizer
from torch.utils.data import DataLoader, Dataset

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model.to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# test
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
labels = tokenizer("example sentence", return_tensors="pt").input_ids

print(input_ids)
print(labels)
"""
# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define your dataset and dataloader
train_dataset =  TextDataset(tokenizer,max_len,max_len)  
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define the training function
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        logits = logits.view(-1, logits.size(-1))
        labels = input_ids.view(-1)
        loss = nn.functional.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Train the model
for epoch in range(3):  # Number of epochs
    train_loss = train_epoch(model, train_dataloader, optimizer, device)
    print(f"Epoch {epoch + 1}: Training loss = {train_loss:.4f}")
"""

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google-t5--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tensor([[13959,  1566,    12,  2968,    10,    37,   629,    19,  1627,     5,
             1]])
tensor([[  644,  4598,   229, 19250,     5,     1]])


'\n# Initialize the optimizer\noptimizer = AdamW(model.parameters(), lr=5e-5)\n\n# Define your dataset and dataloader\ntrain_dataset =  TextDataset(tokenizer,max_len,max_len)  \ntrain_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)\n\n# Define the training function\ndef train_epoch(model, dataloader, optimizer, device):\n    model.train()\n    total_loss = 0\n    for batch in dataloader:\n        input_ids = batch[\'input_ids\'].to(device)\n        attention_mask = batch[\'attention_mask\'].to(device)\n        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)\n        logits = outputs.logits\n        logits = logits.view(-1, logits.size(-1))\n        labels = input_ids.view(-1)\n        loss = nn.functional.cross_entropy(logits, labels)\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n        total_loss += loss.item()\n    return total_loss / len(dataloader)\n\n# Train the model\nfor epoch in range

In [6]:
# Save the model
model.save_pretrained('t5-with-projection')
tokenizer.save_pretrained('t5-with-projection')

('t5-with-projection\\tokenizer_config.json',
 't5-with-projection\\special_tokens_map.json',
 't5-with-projection\\spiece.model',
 't5-with-projection\\added_tokens.json')